# Advanced Lane Finding Project

## Imports

In [1]:
import pickle
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os


# Load saved pickle file to read mtx and dist
cal_pickle = pickle.load(open("camera_cal/cal_pickle.p", "rb"))
mtx = cal_pickle["mtx"]
dist = cal_pickle["dist"]

## Define Sobel & color threshold funtions

In [4]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Calculate the magnitude
    abs_sobel = np.sqrt(sobelx**2 + sobely**2)
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    sobel_scale = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a binary mask where mag thresholds are met
    binary_output = np.zeros_like(sobel_scale)
    binary_output[(sobel_scale >= mag_thresh[0]) & (sobel_scale <= mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    direct_grad = np.arctan2(abs_sobely, abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    binary_output = np.zeros_like(direct_grad)
    binary_output[(direct_grad >= thresh[0]) & (direct_grad <= thresh[1])] = 1
    # 6) Return this mask as your binary_output image

def color_threshold(img, threshs=(0, 255), threshv=(0, 255)):
    
    # Apply the following steps to img
    # 1) Convert to HLS color space
    HLS = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # 2) Apply a threshold to the S channel
    S = HLS[:,:,2]
    binary_S = np.zeros_like(S)
    binary_S[(S > threshs[0]) & (S <= threshs[1])] = 1
    # 3) Covert to HSV color space
    HSV = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # 4)Apply a threshold to the V channel
    V = HSV[:,:,2]
    binary_V = np.zeros_like(V)
    binary_V[(V > threshv[0]) & (V <= threshv[1])] = 1
    # 5) Return a binary image of threshold result
    binary_output = np.zeros_like(S)
    binary_output[(binary_S == 1) & (binary_V == 1)] = 1
    return binary_output 

## Define required functions for finding lanes and fitting polynomials to these lanes

In [5]:
def hist(img): #Find histogram of warped binary image
    bottom_half = img[img.shape[0]//2:,:]
    histogram = np.sum(bottom_half, axis=0)
    return histogram

def find_lane_pixels(binary_warped, histogram, nwindows=9, margin=100, minpix=50):
    # Create an output image for visualization
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    #Find peak points of histogram to determine left and right lane starting points
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # Set height of the windows
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify non-zero pixels of binary warped image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in windows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # Create empty list to receive left and right lane pixels
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        # Define window boundaries
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        #Draw the windows on the image
        cv2.rectangle(out_img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high),
                     (0,255,0), 2)
        cv2.rectangle(out_img, (win_xright_low, win_y_low), (win_xright_high, win_y_high),
                     (0,255,0), 2)
        
        # Identify the nonzero pixels in the windows
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
   
    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    
        
    # Extract left and right lane pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
        
    return leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, out_img

def fit_polynomial(binary_warped, leftx, lefty, rightx, righty, out_img, vis=False):

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    if vis:   
        ## Visualization ##
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]
    
        draw_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)   # needs to be int32 and transposed
        cv2.polylines(out_img, [draw_points_left], False, (0,255,255), 4)  # args: image, points, closed, color
        draw_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)   # needs to be int32 and transposed
        cv2.polylines(out_img, [draw_points_right], False, (0,255,255), 4)  # args: image, points, closed, color

    return out_img, ploty, left_fit, right_fit

def search_around_poly(binary_warped, margin=100, previous_fit=None):
    if previous_fit is None:
        histogram = hist(binary_warped)
        return find_lane_pixels(binary_warped, histogram, nwindows=9, margin=100, minpix=50)
    
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_fit = previous_fit[0]
    right_fit = previous_fit[1]
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] 
    - margin)) & (nonzerox < (left_fit[0]*nonzeroy**2 + left_fit[1]*nonzeroy + left_fit[2] 
    + margin)))
                    
    right_lane_inds = ((nonzerox > (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] 
    - margin)) & (nonzerox < (right_fit[0]*nonzeroy**2 + right_fit[1]*nonzeroy + right_fit[2] 
    + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    out_img, ploty, left_fit, right_fit = fit_polynomial(binary_warped, leftx, lefty, rightx, righty, out_img, vis=False)
    
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    ## Visualization ##
    
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    draw_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)   # needs to be int32 and transposed
    cv2.polylines(result, [draw_points_left], False, (0,255,0), 4)  # args: image, points, closed, color
    draw_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)   # needs to be int32 and transposed
    cv2.polylines(result, [draw_points_right], False, (255,255,255), 4)  # args: image, points, closed, color
    
    return left_fit, right_fit, leftx, lefty, rightx, righty, ploty, left_lane_inds, right_lane_inds, result
    

## Define functions to determine radius of curvature & car offset

In [6]:
def measure_curvature_real(ploty, left_fit, right_fit):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Fit a second order polynomial to pixel positions in each fake lane line
    leftx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    rightx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)
    
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Calculation of Radius of curvature
    left_dxdy = 2*left_fit_cr[0]*ploty[-1]*ym_per_pix + left_fit_cr[1]
    left_dxdy2 =  2*left_fit_cr[0]
    right_dxdy = 2*right_fit_cr[0]*ploty[-1]*ym_per_pix + right_fit_cr[1]
    right_dxdy2 =  2*right_fit_cr[0]
    left_curverad = ((1 + left_dxdy**2)**(3/2)) / abs(left_dxdy2)  ## Implement the calculation of the left line here
    right_curverad = ((1 + right_dxdy**2)**(3/2)) / abs(right_dxdy2)  ## Implement the calculation of the right line here
    
    return left_curverad, right_curverad

def car_offset(img, ploty, left_fit, right_fit):
    # Define conversions in x from pixels space to meters
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    # Calculate x-position of the camera center in pixels
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    camera_mid = (left_fitx[-1] + right_fitx[-1])/2
    
    # Calculate x-poisiton of image mid-point in pixels
    img_mid = img.shape[1]//2
    
    # Calculate horizontal offset of car in meters
    offsetx = (img_mid - camera_mid)*xm_per_pix
    
    # Determine the where the car wrt. the lane
    if offsetx <= 0:
        offset_pos = 'right'
    else:
        offset_pos = 'left'
    
    return np.absolute(offsetx), offset_pos
    

## Define function for visualization

In [7]:
def draw_lane_with_metrics(img, warped_img, ploty, left_fit, right_fit, Minv, left_curverad, right_curverad, offsetx, offset_pos):
    # Obtain x-positions of lanes
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Create image to draw lanes
    zero_warp = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((zero_warp, zero_warp, zero_warp))
    
    left_points = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_points = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    points = np.hstack((left_points, right_points))
    
    # Draw the lane on the color warped image
    cv2.fillPoly(color_warp, np.int_([points]), (0,255, 0))
    cv2.polylines(color_warp, np.int32([left_points]), isClosed=False, color=(255,0,0), thickness=15)
    cv2.polylines(color_warp, np.int32([right_points]), isClosed=False, color=(0,0,255), thickness=15)
    
    # Inverse perspective transform for color warped image with lanes
    new_warp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0]))
    
    #Combine warped image with original image
    result = cv2.addWeighted(img, 1, new_warp, 0.3, 0)
    
    # Add radius of curvatures of lanes and car ofset texts on the image
    result_with_metrics = result.copy()
    cv2.putText(result_with_metrics, 'Radius of Curvature of Left Lane = '+ str(round(left_curverad, 1)) +'(m)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(result_with_metrics, 'Radius of Curvature of Right Lane = '+ str(round(right_curverad, 1)) +'(m)', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.putText(result_with_metrics, 'Vehicle is '+ str(round(offsetx, 1)) + 'm ' + offset_pos + ' of center' , (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    return result_with_metrics

## Pipeline for test images

In [8]:
images = glob.glob('./test_images/test*.jpg')
for fname in images:
    # Read each image in for loop
    img = cv2.imread(fname)
    #Undistort image
    img = cv2.undistort(img, mtx, dist, None, mtx)
    #Save undistorted test images
    file_name = (fname.split('/')[-1]).split('.')[0] + '_undist.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), img)
    
    # Thresholding to create binary image
    binary_sx = abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(15, 100)) # Applying Sobel thresholding in x direction
    binary_sy = abs_sobel_thresh(img, orient='y', sobel_kernel=3, thresh=(25, 100)) # Applying Sobel thresholding in x direction
    binary_color = color_threshold(img, threshs=(100, 255), threshv=(50, 255)) # Applying color threshold in S & V channels 
    combined_binary = np.zeros_like(img[:,:,0])
    combined_binary[(binary_sx == 1) & (binary_sy == 1) | (binary_color == 1)] = 255 # Combined threshold
    
    #Save binary test images
    file_name =  (fname.split('/')[-1]).split('.')[0] + '_binary.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), combined_binary)
    
    # Perspective transform
    xsize = img.shape[1]
    ysize = img.shape[0]
    
    src = np.float32([[xsize*0.46,ysize*0.62], [xsize*0.15,ysize], [xsize*0.88,ysize], [xsize*0.54,ysize*0.62]]) #For project video
    #src = np.float32([[xsize*0.42,ysize*0.72],[xsize*0.17,ysize],[xsize*0.80,ysize],[xsize*0.60,ysize*0.72]]) #For challange video
    
    offset = 300
    dst = np.float32([[offset, 0], [offset, ysize], [xsize-offset, ysize], [xsize-offset, 0]])
    
    
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(combined_binary, M, (xsize, ysize), flags=cv2.INTER_LINEAR)
    
    #Save binary wraped test images
    file_name = (fname.split('/')[-1]).split('.')[0] + '_binary_wraped.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), warped)
    
    # Finding the histogram of binary warped images
    histogram = hist(warped)
    # Find lane pixels 
    leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, output_img = find_lane_pixels(warped, histogram, nwindows=9, margin=100, minpix=50)

    lane_img, ploty, left_fit, right_fit = fit_polynomial(warped, leftx, lefty, rightx, righty, output_img, vis=True)
    fitted_lanes = [left_fit, right_fit]
    
    #Save test images with lanes
    file_name = (fname.split('/')[-1]).split('.')[0] + '_fitlane.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), lane_img)
    
    left_fit, right_fit, leftx, lefty, rightx, righty, ploty, left_lane_inds, right_lane_inds, binary_result = search_around_poly(warped, margin=100, previous_fit=fitted_lanes) # Finding lanes using previous fitted lanes
    
    #Save test images with lanes
    file_name = (fname.split('/')[-1]).split('.')[0] + '_binary_result.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), binary_result)
    
    # Determine radius of curvatures of the lanes and car offset
    left_curverad, right_curverad = measure_curvature_real(ploty, left_fit, right_fit)
    offsetx, offset_pos = car_offset(img, ploty, left_fit, right_fit)
    
    # Image with metrics and lanes
    result_with_metrics = draw_lane_with_metrics(img, warped, ploty, left_fit, right_fit, Minv, left_curverad, right_curverad, offsetx, offset_pos)
    
    #Save the test results
    file_name =  (fname.split('/')[-1]).split('.')[0] + '_output.jpg'
    cv2.imwrite(os.path.join('./output_images', file_name), result_with_metrics)
    
    


In [ ]:
img = cv2.imread("test_images/test7.jpg")

binary_sx = abs_sobel_thresh(img, orient='x', sobel_kernel=7, thresh=(100, 200)) # Applying Sobel thresholding in x direction
binary_sy = abs_sobel_thresh(img, orient='y', sobel_kernel=7, thresh=(50, 200)) # Applying Sobel thresholding in x direction
binary_mag = mag_thresh(img, sobel_kernel=3, mag_thresh=(75, 150))
#binary_color = color_threshold(img, threshs=(100, 255), threshv=(50, 255)) # Applying color threshold in S & V channels 
binary_color = color_threshold(img, threshs=(50,255), threshv=(50,255))
combined_binary = np.zeros_like(img[:,:,0])
#combined_binary[(binary_sx == 1) & (binary_sy == 1) | (binary_color == 1)] = 255 # Combined threshold
combined_binary[(binary_color == 1) | (binary_mag == 1)] = 255

# Perspective transform
xsize = img.shape[1]
ysize = img.shape[0]
#src = np.float32([[xsize*0.46,ysize*0.62], [xsize*0.15,ysize], [xsize*0.88,ysize], [xsize*0.54,ysize*0.62]]) #For project video   
src = np.float32([[xsize*0.42,ysize*0.72],[xsize*0.17,ysize],[xsize*0.80,ysize],[xsize*0.60,ysize*0.72]]) # For challenge video
#src = np.float32([[xsize*0.47,ysize*0.62], [xsize*0.18,ysize], [xsize*0.91,ysize], [xsize*0.56,ysize*0.62]])
offset = 300
dst = np.float32([[offset, 0], [offset, ysize], [xsize-offset, ysize], [xsize-offset, 0]])
# Given src and dst points, calculate the perspective transform matrix
M = cv2.getPerspectiveTransform(src, dst)
Minv = cv2.getPerspectiveTransform(dst, src)
# Warp the image using OpenCV warpPerspective()
warped = cv2.warpPerspective(combined_binary, M, (xsize, ysize), flags=cv2.INTER_LINEAR)

pts = np.array([[xsize*0.42,ysize*0.72],[xsize*0.17,ysize],[xsize*0.80,ysize],[xsize*0.60,ysize*0.72]], np.int32)
pts = pts.reshape((-1,1,2))
result = cv2.polylines(img,[pts],True,(0,0,255),4)
r,g,b = cv2.split(result)
result = cv2.merge((b,g,r))

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
f.subplots_adjust(hspace = .2, wspace=.05)
ax1.imshow(result)
ax1.set_title('Box', fontsize=30)
ax2.imshow(warped, cmap='gray')
ax2.set_title('binary', fontsize=30)

## Define class to track lane detections

In [ ]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #offset position
        self.offset_pos = None
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    def add_lane(self, fit):
        if fit is not None: # If the lane is detected
            if self.best_fit is not None: # If best fit is already detected
                self.diffs = fit - self.best_fit # Compare the new fit with best fit
            if (abs(self.diffs[0]) > 0.001 or abs(self.diffs[1] > 1.0) or abs(self.diffs[2] > 100.0)
                and len(self.current_fit) > 0): # Sanity check fails
                self.detected = False 
            else:
                self.detected = True
                self.current_fit.append(fit)
                if len(self.current_fit) > 3: # Hold only last 3 fit
                    self.current_fit = self.current_fit[len(self.current_fit)-3:]
                self.best_fit = fit
                #self.best_fit = np.average(self.current_fit[1:], axis=0)
        else: # If the lane is not detected
            self.detected = False
            if len(self.current_fit) > 3:
                self.current_fit = self.current_fit[:len(self.current_fit)-1] # Remove last fit
            if len(self.current_fit) > 3:
                self.best_fit = np.average(self.current_fit[1:], axis=0)

## Define complete lane finding pipeline for video processing

In [ ]:
def process_image(img):
    #Undistort image
    img = cv2.undistort(img, mtx, dist, None, mtx)
    # Thresholding to create binary image
    #binary_sx = abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(15, 100)) # Applying Sobel thresholding in x direction
    #binary_sy = abs_sobel_thresh(img, orient='y', sobel_kernel=3, thresh=(25, 100)) # Applying Sobel thresholding in x direction
    #binary_color = color_threshold(img, threshs=(100, 255), threshv=(50, 255)) # Applying color threshold in S & V channels
    #combined_binary = np.zeros_like(img[:,:,0])
    #combined_binary[(binary_sx == 1) & (binary_sy == 1) | (binary_color == 1)] = 255 # Combined threshold
    
    binary_sx = abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(15, 255)) # Applying Sobel thresholding in x direction
    binary_sy = abs_sobel_thresh(img, orient='y', sobel_kernel=3, thresh=(25, 255)) # Applying Sobel thresholding in x direction
    binary_color = color_threshold(img, threshs=(100, 255), threshv=(50, 255)) # Applying color threshold in S & V channels 
    combined_binary = np.zeros_like(img[:,:,0])
    combined_binary[(binary_sx == 1) & (binary_sy == 1) | (binary_color == 1)] = 255 # Combined threshold
    
    # Perspective transform
    xsize = img.shape[1]
    ysize = img.shape[0]
    #src = np.float32([[xsize*0.46,ysize*0.62], [xsize*0.15,ysize], [xsize*0.88,ysize], [xsize*0.54,ysize*0.62]]) #For project video
    src = np.float32([[xsize*0.42,ysize*0.72],[xsize*0.17,ysize],[xsize*0.80,ysize],[xsize*0.60,ysize*0.72]]) #For challange video
    #src = np.float32([[xsize*0.47,ysize*0.62], [xsize*0.18,ysize], [xsize*0.91,ysize], [xsize*0.56,ysize*0.62]])
    offset = 300
    dst = np.float32([[offset, 0], [offset, ysize], [xsize-offset, ysize], [xsize-offset, 0]])
   
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(combined_binary, M, (xsize, ysize), flags=cv2.INTER_LINEAR)
    
    # If left and right lanes were detected in last frame, use search_around_poly function, otherwise use find_lane_pixels and fit_polynomial functions
    if left_lane.detected and right_lane.detected:
        fitted_lanes = [left_lane.best_fit, right_lane.best_fit]
        left_fit, right_fit, leftx, lefty, rightx, righty, ploty, left_lane_inds, right_lane_inds, binary_result = search_around_poly(warped, margin=100, previous_fit=fitted_lanes)
    else:
        histogram = hist(warped)
        leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, output_img = find_lane_pixels(warped, histogram, nwindows=9, margin=100, minpix=50)
        lane_img, ploty, left_fit, right_fit = fit_polynomial(warped, leftx, lefty, rightx, righty, output_img, vis=False)
     
    ## Sanity Checks
    # Checking that they are separated by approximately the right distance horizontally
    if left_fit is not None and right_fit is not None:
        height = img.shape[0]
        left_lane_x = left_fit[0]*height**2 + left_fit[1]*height + left_fit[2]
        right_lane_x = right_fit[0]*height**2 + right_fit[1]*height + right_fit[2]
        diff_x = np.absolute(left_lane_x - right_lane_x)
        if np.absolute(diff_x - 600) > 200:
            print("1")
            print(np.absolute(diff_x - 600))
            left_fit = None
            right_fit = None
    
    # Checking that they have similar curvature
    if left_fit is not None and right_fit is not None:
        # Determine radius of curvatures of the lanes and car offset
        left_curverad, right_curverad = measure_curvature_real(ploty, left_fit, right_fit)
        diff_curverad = np.absolute(left_curverad - right_curverad)
        if diff_curverad > 10000:
            print("2")
            left_fit = None
            rigth_fit = None
    
    """
    # Checking that they are roughly parallel
    if left_fit is not None and right_fit is not None:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        diff_fitx = np.absolute(left_fitx - right_fitx)
        if np.max(np.absolute(diff_fitx - np.mean(diff_fitx))) > 200:
            print("3")
            left_fit = None
            right_fit = None 
    """ 
    # Add fits to array
    left_lane.add_lane(left_fit)
    right_lane.add_lane(right_fit)
    
    if left_lane.best_fit is not None and right_lane.best_fit is not None:
        left_lane.radius_of_curvature, right_lane.radius_of_curvature = measure_curvature_real(ploty, left_lane.best_fit, right_lane.best_fit)
        left_lane.offsetx, left_lane.offset_pos = car_offset(img, ploty, left_lane.best_fit, right_lane.best_fit)
        # Image with metrics and lanes
        output_img = draw_lane_with_metrics(img, warped, ploty, left_lane.best_fit, right_lane.best_fit, Minv,
                                            left_lane.radius_of_curvature, right_lane.radius_of_curvature,
                                            left_lane.offsetx, left_lane.offset_pos)
    else:
        output_img = np.copy(img)
    return output_img
    
    

In [ ]:
img = cv2.imread("test_images/tes5.jpg")
left_lane = Line()
right_lane = Line()


output_img = process_image(img)
r,g,b = cv2.split(output_img)
output_img = cv2.merge((b,g,r))
plt.imshow(output_img)

In [ ]:
current_fit = [np.array([False])]
current_fit.append(left_fit)
current_fit.append(right_fit)
best_fit = np.average(current_fit[1:], axis=0)
print(current_fit)
print(best_fit)

## Process project_video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Create left and right lane objects
left_lane = Line()
right_lane = Line()

project_video_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
#clip1 = VideoFileClip("project_video.mp4").subclip(10,30)
clip1 = VideoFileClip("project_video.mp4")
project_video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time project_video_clip.write_videofile(project_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_video_output))

## Process challange_video

In [ ]:
# Create left and right lane objects
left_lane = Line()
right_lane = Line()

challenge_video_output = 'challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)
#clip1 = VideoFileClip("challenge_video.mp4")
challenge_video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time challenge_video_clip.write_videofile(challenge_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_video_output))

## Process harder_challenge video

In [ ]:
# Create left and right lane objects
left_lane = Line()
right_lane = Line()

harder_challenge_video_output = 'harder_challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
clip1 = VideoFileClip("challenge_video.mp4").subclip(0,5)
#clip1 = VideoFileClip("harder_challenge_video.mp4")
harder_challenge_video_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time harder_challenge_video_clip.write_videofile(challenge_video_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_video_output))